# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing]. 
Please download the data and place it in your data_path at "./data."

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

In [4]:
data_path = './data/'
if not os.path.exists(data_path + 'train_data.npy') or not os.path.exists(data_path + 'test_data.npy') or not os.path.exists(data_path + 'train_label.npy') or not os.path.exists(data_path + 'test_label.npy'):
        print("ERROR: Dataset not found. Please download the dataset from 'https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing'")
        print("\n\n")
        raise FileNotFoundError("Please download the dataset from 'https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing'")

ERROR: Dataset not found. Please download the dataset from 'https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing'





FileNotFoundError: Please download the dataset from 'https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing'

In [4]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data) 
y_train_tensor = torch.LongTensor(train_label) 
x_test_tensor = torch.Tensor(test_data)
y_test_tensor = torch.LongTensor(test_label)

In [5]:
# visualize the training labels
train_label

array([1, 1, 0, ..., 3, 0, 0])

In [6]:
print("Cuda is available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

Cuda is available:  True


In [7]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

In [8]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes, hidden_units=[256, 128, 64]):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, hidden_units[0]), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            # nn.ReLU(), # Use ReLu function for NN training 
            nn.Softplus(),
            nn.Linear(hidden_units[0], hidden_units[1]), # 256 NN units to 128 units
            # nn.ReLU(),
            nn.Softplus(),
            nn.Linear(hidden_units[1], hidden_units[2]),#  128 NN units to 64 units
            # nn.ReLU()
            nn.Softplus(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden_units[2], num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        
        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [9]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes, hidden_units=[1024, 512, 256]).to(device) 
criterion = nn.NLLLoss() # Use NLLLoss function to optimize
optimizer = optim.Adam(model.parameters(), lr=0.0001) # Setting parameters learning rate = 0.001

/home/fzhao12/.cache/pypoetry/virtualenvs/bci-project-W8FRcYYV-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
num_epochs = 500 # setting training epochs (Number of training iterations)
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader: 
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/500, Loss: 1.660423755645752
Epoch 2/500, Loss: 1.4420137405395508
Epoch 3/500, Loss: 1.3515323400497437
Epoch 4/500, Loss: 1.1079072952270508
Epoch 5/500, Loss: 0.8526711463928223
Epoch 6/500, Loss: 0.7015528082847595
Epoch 7/500, Loss: 0.5894370079040527
Epoch 8/500, Loss: 0.4076138436794281
Epoch 9/500, Loss: 0.15064243972301483
Epoch 10/500, Loss: 0.1570926308631897
Epoch 11/500, Loss: 0.07666055858135223
Epoch 12/500, Loss: 0.046869996935129166
Epoch 13/500, Loss: 0.05492643266916275
Epoch 14/500, Loss: 0.044283028692007065
Epoch 15/500, Loss: 0.039010848850011826
Epoch 16/500, Loss: 0.030360883101820946
Epoch 17/500, Loss: 0.02121611498296261
Epoch 18/500, Loss: 0.03355458006262779
Epoch 19/500, Loss: 0.1021791398525238
Epoch 20/500, Loss: 0.03338821604847908
Epoch 21/500, Loss: 0.07286013662815094
Epoch 22/500, Loss: 0.04369252920150757
Epoch 23/500, Loss: 0.22485153377056122
Epoch 24/500, Loss: 0.05284222960472107
Epoch 25/500, Loss: 0.15010812878608704
Epoch 26/500, Lo

In [12]:
def evaluate_model(model):
    model.eval() # Evaluate your model
    correct = 0
    total = 0

    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # print the predited label and the true label
    print(predicted)
    print(labels)

In [13]:
evaluate_model(model)
# save the model with num_epochs suffix to 'trained_models' directory
file_name = f'trained_models/eeg_autoencoder_classifier_{num_epochs}.pth'
torch.save(model.state_dict(), file_name)

Test Accuracy: 57.81%
tensor([2, 4, 0, 3, 0, 0, 3, 2, 2, 4, 3, 1, 3, 2, 0, 4, 4, 4, 3, 3, 4, 2, 2, 1,
        3, 1, 3, 0, 2, 3, 1, 3, 2, 3, 4, 3, 4, 4, 3, 3, 3, 1, 3, 0, 2, 3, 2, 1,
        1, 4, 0, 2, 2, 4, 0, 3, 2, 3, 2, 1, 2, 4, 4, 3], device='cuda:0')
tensor([2, 4, 1, 4, 4, 1, 4, 2, 1, 0, 3, 0, 3, 2, 3, 4, 4, 4, 0, 4, 1, 0, 2, 1,
        3, 1, 0, 4, 2, 3, 1, 2, 2, 0, 4, 3, 2, 4, 2, 0, 3, 3, 1, 0, 2, 3, 1, 0,
        1, 4, 0, 2, 1, 4, 0, 1, 2, 3, 4, 1, 2, 4, 3, 3], device='cuda:0')


In [15]:
# load the model from the file
model = EEGAutoencoderClassifier(num_classes=5, hidden_units=[1024, 512, 256]).to(device)
model.load_state_dict(torch.load(file_name))
evaluate_model(model)

Test Accuracy: 57.81%
tensor([2, 4, 0, 3, 0, 0, 3, 2, 2, 4, 3, 1, 3, 2, 0, 4, 4, 4, 3, 3, 4, 2, 2, 1,
        3, 1, 3, 0, 2, 3, 1, 3, 2, 3, 4, 3, 4, 4, 3, 3, 3, 1, 3, 0, 2, 3, 2, 1,
        1, 4, 0, 2, 2, 4, 0, 3, 2, 3, 2, 1, 2, 4, 4, 3], device='cuda:0')
tensor([2, 4, 1, 4, 4, 1, 4, 2, 1, 0, 3, 0, 3, 2, 3, 4, 4, 4, 0, 4, 1, 0, 2, 1,
        3, 1, 0, 4, 2, 3, 1, 2, 2, 0, 4, 3, 2, 4, 2, 0, 3, 3, 1, 0, 2, 3, 1, 0,
        1, 4, 0, 2, 1, 4, 0, 1, 2, 3, 4, 1, 2, 4, 3, 3], device='cuda:0')
